<a href="https://colab.research.google.com/github/Aleena24/real-image-SRGAN/blob/main/research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data loading

In [34]:
import os
import numpy as np
import cv2

# Define the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/research_dataset/BSD100'

#Image Preprocessing

In [35]:
# Define the image size
image_size = (256, 256)

# Function to load and preprocess images from the dataset directory
def load_and_preprocess_dataset(dataset_dir, image_size):
    X_train = []
    y_train = []

    for filename in os.listdir(dataset_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(dataset_dir, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, image_size)  # Resize image to desired size

            # Assuming filename contains information about the label or target
            # Extract label or target from the filename and preprocess if needed
            label = extract_label_from_filename(filename)

            X_train.append(img)
            y_train.append(label)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train

# Function to extract label or target from filename (modify as per your dataset)
def extract_label_from_filename(filename):
    # Implement logic to extract label from filename
    # For example, split filename and extract label from the filename
    label = filename.split('_')[0]  # Example: if filename is 'cat_001.jpg', label will be 'cat'
    return label

# Load and preprocess the dataset
X_train, y_train = load_and_preprocess_dataset(dataset_dir, image_size)

# Optionally, you can normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255.0


# Generator Network

In [36]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_generator(input_shape=(64, 64, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(64, kernel_size=9, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)

    # Residual blocks
    for _ in range(16):
        residual = x

        x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.Conv2D(64, kernel_size=3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.add([x, residual])

    # Decoder
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    outputs = layers.Conv2D(3, kernel_size=9, padding='same', activation='tanh')(x)

    generator = Model(inputs, outputs, name='generator')
    return generator

# Test the generator model
generator = build_generator()
generator.summary()


Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_102 (Conv2D)         (None, 64, 64, 64)           15616     ['input_20[0][0]']            
                                                                                                  
 conv2d_103 (Conv2D)         (None, 64, 64, 64)           36928     ['conv2d_102[0][0]']          
                                                                                                  
 conv2d_104 (Conv2D)         (None, 64, 64, 64)           36928     ['conv2d_103[0][0]']          
                                                                                          

#Discriminator Network

In [37]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_discriminator(input_shape=(256, 256, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    # Convolutional layers
    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.25)(x)

    # Flatten layer
    x = layers.Flatten()(x)

    # Output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)

    discriminator = Model(inputs, outputs, name='discriminator')
    return discriminator

# Test the discriminator model
discriminator = build_discriminator()
discriminator.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_138 (Conv2D)         (None, 128, 128, 64)      1792      
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 128, 128, 64)      0         
                                                                 
 dropout_12 (Dropout)        (None, 128, 128, 64)      0         
                                                                 
 conv2d_139 (Conv2D)         (None, 64, 64, 128)       73856     
                                                                 
 batch_normalization_74 (Ba  (None, 64, 64, 128)       512       
 tchNormalization)                                               
                                                     

#Loss function

# generator loss

In [38]:
import tensorflow as tf

def generator_loss(fake_output, reconstructed_output):
    # Calculate mean absolute error between the generated image and the target image
    return tf.reduce_mean(tf.abs(fake_output - reconstructed_output))


# discriminator loss

In [39]:
import tensorflow as tf

def discriminator_loss(real_output, fake_output):
    # Calculate the binary cross-entropy loss for the discriminator
    real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(real_output), logits=real_output))
    fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(fake_output), logits=fake_output))
    total_loss = real_loss + fake_loss
    return total_loss


# Initialization

In [40]:
# Initialize the generator
generator = build_generator()

# Initialize the discriminator
discriminator = build_discriminator()


# optimizer

In [41]:
# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

#Compilation

In [42]:
# Compile the models
generator.compile(optimizer=optimizer, loss=generator_loss)
discriminator.compile(optimizer=optimizer, loss=discriminator_loss)

# Combined model (GAN)
input_lr = tf.keras.Input(shape=(64, 64, 3))  # Use the same shape as the output of the generator
input_hr = tf.keras.Input(shape=(256, 256, 3))

sr_image = generator(input_lr)
discriminator.trainable = False
validity = discriminator(sr_image)
gan = Model(inputs=[input_lr, input_hr], outputs=[validity, sr_image])
gan.compile(optimizer=optimizer, loss=[discriminator_loss, generator_loss])

ValueError: Exception encountered when calling layer "discriminator" (type Functional).

Input 0 of layer "dense_5" is incompatible with the layer: expected axis -1 of input shape to have value 262144, but received input with shape (None, 65536)

Call arguments received by layer "discriminator" (type Functional):
  • inputs=tf.Tensor(shape=(None, 128, 128, 3), dtype=float32)
  • training=None
  • mask=None

# Training

In [ ]:
# Train the GAN
epochs = 100
batch_size = 16

for epoch in range(epochs):
    for i in range(len(X_train) // batch_size):
        idx = np.random.randint(0, len(X_train), batch_size)
        real_images = y_train[idx]
        low_res_images = X_train[idx]

        fake_images = generator.predict(low_res_images)

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_images, valid)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = gan.train_on_batch([low_res_images, real_images], [valid, real_images])

        print(f"Epoch: {epoch}, Step: {i}/{len(X_train) // batch_size}, D Loss: {d_loss}, G Loss: {g_loss}")

super_resolved_images = generator.predict(X_test)
